In [1]:
%load_ext rpy2.ipython

In [ ]:
###
 # @author [Wankun Deng]
 # @email [dengwankun@gmail.com]
 # @create date 2023-06-30 13:17:49
 # @modify date 2023-07-07 14:09:09
 # @desc [description]
###
import os
import sys
import random
import numpy as np
import pandas as pd
from multiprocessing import Pool
from scipy.stats import mannwhitneyu

sys.argv=['th','../data/all_datasets','../www/mysql','../../universal_data/rmsk/rmsk_GRCh38.txt']
input_path=sys.argv[1]
output_path=sys.argv[2]
rmsk_f=sys.argv[3]

file_list=[x for x in os.listdir(input_path) if x.endswith('.cell_exp.txt')]
rmsk=pd.read_table(rmsk_f,sep='\t',header=None)
rtes=rmsk[rmsk[11].isin(['LINE','SINE','LTR'])][10].unique()
datasets=list(set([x.split('.')[0] for x in file_list]))
datasets.sort()
cell_types=['Ex','Ast','Mic','OPC','Oli','In']
def get_dataset(dataset):
    print(dataset)
    randint=random.randint(0,1000)
    print(f'loading data: {dataset}_{randint}')
    results=[]
    dt_ls=[x for x in file_list if x.startswith(dataset)]
    cell_exp=pd.read_table(os.path.join(input_path,dt_ls[0]),index_col=0)
    if len(dt_ls) >1:
        cell_umap=pd.read_table(f'{input_path}/{dataset}.1.cell_umap.txt',index_col=0)
        for i in range(1,len(dt_ls)):
            cell_exp=pd.concat([cell_exp,pd.read_table(f'{input_path}/'+dt_ls[i],index_col=0)])
            cell_umap=pd.concat([cell_umap,pd.read_table(f'{input_path}/{dataset}.{i}.cell_umap.txt',index_col=0)])
    else:
        cell_umap=pd.read_table(f'{input_path}/'+dataset+'.cell_umap.txt',index_col=0)
    
    cell_umap['predicted.celltype'] = cell_umap['predicted.celltype'].replace(
        'Opc', 'OPC')
    for i in range(cell_umap.shape[0]):
        if cell_umap.iloc[i,1] =='Stage_0':
            cell_umap.iloc[i,1]='Control'
        if cell_umap.iloc[i,1] !='Control':
            cell_umap.iloc[i,1]=cell_umap.iloc[i,7].split('_')[0]

    cell_exp=cell_exp.loc[:,[x for x in cell_exp.columns if x in rtes]]
    ## disease v.s. control
    print('extracting disease v.s. control')
    disease=dataset.split('_')[0]
    for cell in cell_types:
        disease_list=cell_umap.loc[(cell_umap['predicted.celltype']==cell) & (cell_umap['Diagnosis']!='Control'),:].index
        control_list=cell_umap.loc[(cell_umap['predicted.celltype']==cell) & (cell_umap['Diagnosis']=='Control'),:].index
        for te in rtes:
            if te in cell_exp.columns:
                disease_exp=np.exp(cell_exp.loc[disease_list,te])
                control_exp=np.exp(cell_exp.loc[control_list,te])
                pval=mannwhitneyu(disease_exp,control_exp)[1]
                c1_median=np.mean(disease_exp)
                c2_median=np.mean(control_exp)
                pce1=np.count_nonzero(disease_exp>1)/(1.0*len(disease_exp))
                pce2=np.count_nonzero(control_exp>1)/(1.0*len(control_exp))
                fc=np.log2(np.mean(disease_exp)/np.mean(control_exp))
                results.append([te,f'{disease} v.s. Control',dataset,cell,'NA',np.log(c1_median),np.log(c2_median),fc,pval,pce1,pce2,len(disease_exp),len(control_exp)])
    ## cell v.s. cell
    print('extracting cell v.s. cell')
    for cell1 in cell_types:
        for cell2 in cell_types:
            if not cell1==cell2:
                for condition in [disease,'Control']:
                    cell1_list=cell_umap.loc[(cell_umap['predicted.celltype']==cell1) & (cell_umap['Diagnosis']==condition),:].index
                    cell2_list=cell_umap.loc[(cell_umap['predicted.celltype']==cell2) & (cell_umap['Diagnosis']==condition),:].index
                    for te in rtes:
                        if te in cell_exp.columns:
                            cell1_exp=np.exp(cell_exp.loc[cell1_list,te])
                            cell2_exp=np.exp(cell_exp.loc[cell2_list,te])
                            pval=mannwhitneyu(cell1_exp,cell2_exp)[1]
                            c1_median=np.mean(cell1_exp)
                            c2_median=np.mean(cell2_exp)
                            fc=np.log2(np.mean(cell1_exp)/np.mean(cell2_exp))
                            pce1=np.count_nonzero(cell1_exp>1)/(1.0*len(cell1_exp))
                            pce2=np.count_nonzero(cell2_exp>1)/(1.0*len(cell2_exp))
                            results.append([te,f'{cell1} v.s. {cell2}',dataset,'NA',condition,np.log(c1_median),np.log(c2_median),fc,pval,pce1,pce2,len(cell1_exp),len(cell2_exp)])
    return results

pool = Pool(20)
print(datasets)
ret=pool.map(get_dataset,datasets)
pool.close()
pool.join()

final_result=[]
for i in ret:
    final_result.extend(i)
final_result=pd.DataFrame(final_result,columns=['TE','Comparison','Dataset','Cell','Condition','log2_median1','log2_median2','log2_FC','pval','pce1','pce2','cell1_count','cell2_count'])

te_de=open(f'{output_path}/exp_de.sql','w')

te_de.write('''use scARE;
    DROP TABLE IF EXISTS `EXP_DE`;
    CREATE TABLE `EXP_DE` (
        `TE` varchar(255) NOT NULL,
        `COMPARISON` varchar(255) NOT NULL,
        `DATASET` varchar(255) NOT NULL,
        `CELL` varchar(255) NOT NULL,
        `CONDITION` varchar(255) NOT NULL,
        `MEAN1` float NOT NULL,
        `MEAN2` float NOT NULL,
        `FC` float NOT NULL,
        `PVAL` float NOT NULL,
        `PCE1` float NOT NULL,
        `PCE2` float NOT NULL,
        `CELL1_COUNT` int NOT NULL,
        `CELL2_COUNT` int NOT NULL);
        set autocommit=0;\n''')

count=0
values_list=[]
template='INSERT INTO `EXP_DE` (TE,COMPARISON,DATASET,CELL,`CONDITION`,MEAN1,MEAN2,`FC`,PVAL,PCE1,PCE2,CELL1_COUNT,CELL2_COUNT) VALUES {values};\n'
for i in range(final_result.shape[0]):
    if count==2000:
        te_de.write(template.format(values=','.join(values_list)))
        count=0
        values_list=[]
    row=list(final_result.iloc[i,:])
    row[:5]=['"'+str(x)+'"' for x in row[:5]]
    values_list.append('('+','.join([str(x) for x in row])+')')
    count+=1

if count>0:
    te_de.write(template.format(values=','.join(values_list)))
te_de.write('commit;')
te_de.close()

/tmp/ipykernel_349084/3626175138.py:22: DtypeWarning: Columns (0,1,2,3,4,6,7,8,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  rmsk=pd.read_table(rmsk_f,sep='\t',header=None)


AD_HS_00004AD_HS_00005AD_HS_00001AD_HS_00002AD_HS_00003AD_HS_00007MS_HS_00001AD_HS_00006AD_HS_00008MS_HS_00002PD_HS_00001









loading data: AD_HS_00004_94
loading data: AD_HS_00005_658loading data: AD_HS_00001_279loading data: AD_HS_00002_624loading data: AD_HS_00003_336loading data: AD_HS_00007_562loading data: AD_HS_00008_571loading data: AD_HS_00006_752loading data: MS_HS_00002_166loading data: PD_HS_00001_55loading data: MS_HS_00001_612










['AD_HS_00001', 'AD_HS_00002', 'AD_HS_00003', 'AD_HS_00004', 'AD_HS_00005', 'AD_HS_00006', 'AD_HS_00007', 'AD_HS_00008', 'MS_HS_00001', 'MS_HS_00002', 'PD_HS_00001']
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. control
extracting cell v.s. cell
extracting disease v.s. co

In [ ]:
# ## assign cell type according to ScType tuturial

## load sc type and database
# source(paste0(sctype_path,"/R/gene_sets_prepare.R"))
# source(paste0(sctype_path,"/R/sctype_score_.R"))
# gs_list = gene_sets_prepare(paste0(sctype_path,'/ScTypeDB_full.xlsx'), "Brain" )
# # get cell-type by cell matrix
# es.max = sctype_score(scRNAseqData = scte[["RNA"]]@scale.data, scaled = TRUE, 
#                       gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

# # merge by cluster
# cL_resutls = do.call("rbind", lapply(unique(scte@meta.data$seurat_clusters), function(cl){
#     es.max.cl = sort(rowSums(es.max[ ,rownames(scte@meta.data[scte@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
#     head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(scte@meta.data$seurat_clusters==cl)), 10)
# }))
# sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# # set low-confident (low ScType score) clusters to "unknown"
# sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"


# scte@meta.data$customclassif = ""
# for(j in unique(sctype_scores$cluster)){
#   cl_type = sctype_scores[sctype_scores$cluster==j,]; 
#   scte@meta.data$customclassif[scte@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
# }

# ## save cell type
# cell_types<-scte[['customclassif']]


In [ ]:
%%R
library(stringr)

lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)

load('../data/reference.RData')

sample_meta<-'AD_HS_00001	Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease	20	H.Sapiens	AD	Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex	Braak:0;0;0;2;2;2;2;6;6;6;0;0;0;2;2;2;2;6;6;6	ExcitatoryNeuron;InhibitoryNeuron;Oligodendrocyte;OPC;Astrocyte;Microglia;Endothelia;Pericyte	GSE147528	Single-nucleus RNA-seq	10x Genomics	Male	60;50;71;72;87;91;77;82;72;82;60;50;71;72;87;91;77;82;72;82	SRR11422700;SRR11422701;SRR11422702;SRR11422703;SRR11422704;SRR11422705;SRR11422706;SRR11422707;SRR11422708;SRR11422709;SRR11422710;SRR11422711;SRR11422712;SRR11422713;SRR11422714;SRR11422715;SRR11422716;SRR11422717;SRR11422718;SRR11422719'
split_meta<-unlist(strsplit(sample_meta, '\t'))
samples<-unlist(strsplit(split_meta[14],';'))
disease<-rep(split_meta[5],length(samples))
stage<-unlist(strsplit(unlist(strsplit(split_meta[7],':'))[2],';'))
gender<-rep(split_meta[12],length(samples))
age<-unlist(strsplit(split_meta[13],';'))

sample_meta<-data.frame(samples=samples,disease=disease,stage=stage,gender=gender,age=age)
sample_meta[sample_meta$stage=='0','disease']<-'Control'
rownames(sample_meta)<-sample_meta$samples

# args <- commandArgs(trailingOnly=TRUE)
args<-c('../data/3/scte','../data/3/cell_umap.txt','SRR11422700','/home/wdeng3/workspace/software/sc-type','../../universal_data/rmsk/rmsk_GRCh38.txt')
data_path<-args[1]
out_path<-args[2]
sample_tag<-args[3]
sctype_path<-args[4]
rmsk_path<-args[5]
rmsk<-read.table(rmsk_path,sep='\t',header=F)
to_remove<-unlist(rmsk[!(rmsk$V12 %in% c('LINE','SINE','LTR')),'V11'])
to_remove<-str_replace_all(to_remove,'-','\\.')

reps<-unlist(rmsk[rmsk$V12 %in% c('LINE','SINE','LTR'),'V11'])
reps<-str_replace_all(reps,'-','\\.')

create_seurat<-function(sample_tag){
    ## load data
    scte.data <- t(read.csv(paste0(data_path,'/',sample_tag,'/',sample_tag,'.csv'),check.names=F, row.names = 1))
    # remove repeats from gene expression matrix

    scte.gene<-scte.data[!(row.names(scte.data) %in% to_remove),]
    # normalize and scale data
    scte <- CreateSeuratObject(counts = scte.gene, project = sample_tag, min.cells = 3, min.features = 200)
    # normalize data
    scte[["percent.mt"]] <- PercentageFeatureSet(scte, pattern = "^MT.|^MT-")
    scte <- subset(scte, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
    scte <- SCTransform(scte, vars.to.regress = "percent.mt", verbose = FALSE)

    cell_meta<-scte@meta.data
    cell_meta$disease<-sample_meta[cell_meta$orig.ident,'disease']

    cell_meta$stage<-sample_meta[cell_meta$orig.ident,'stage']
    cell_meta$gender<-sample_meta[cell_meta$orig.ident,'gender']
    cell_meta$age<-sample_meta[cell_meta$orig.ident,'age']
    scte@meta.data<-cell_meta
    scte
}

scte<-create_seurat(sample_tag)

scte <- FindVariableFeatures(scte, selection.method = "vst", nfeatures = 2000)
# scale and run PCA
scte <- ScaleData(scte, features = rownames(scte))
scte <- RunPCA(scte, features = VariableFeatures(object = scte))

sdv<-Stdev(scte,reduction='pca')
sdv<-sdv[sdv>2]
npca<-length(sdv)

scte <- FindNeighbors(scte, dims = 1:npca)
scte <- FindClusters(scte, resolution = 0.8)
scte <- RunUMAP(scte, dims = 1:npca)


scte.anchors <- FindTransferAnchors(reference = reference, query = scte,
    dims = 1:30, reference.reduction = "pca")
scte.query <- MapQuery(anchorset = scte.anchors, reference = reference, query = scte,
    refdata = list(celltype = "broad.cell.type"), reference.reduction = "pca", reduction.model = "umap")
cell_meta<-scte.query@meta.data

cell_meta$UMAP_1<-scte.query@reductions[['umap']]@cell.embeddings[,1]
cell_meta$UMAP_2<-scte.query@reductions[['umap']]@cell.embeddings[,2]
scte.query@meta.data<-cell_meta
write.table(cell_meta,out_path,sep='\t',quote=F,row.names=T)

## get gene expression from scte_query
scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)